# The Quantum Harmonic Oscillator: A Simulation Journey

Welcome! In this notebook, we're going to interactively learn how to simulate the Quantum Harmonic Oscillator (QHO). By the end, you'll be able to solve for its energy levels, visualize its wavefunctions, and even watch them evolve in time, all using Python.

### Why simulating?
Analytic solutions are great, but many real-world quantum systems (like atoms in cavities or superconducting circuits) are too complex for pen and paper. Simulations let us play with these systems to build intuition, plotting helps you get a feel for concepts quickly.

### Prerequisites
We assume you know some basic Python (variables, loops, functions). If you're rusty, check out [ThinkPython](https://allendowney.github.io/ThinkPython/). We'll teach you the libraries (`numpy`, `matplotlib`, `qutip`) as we go.

---

## Part 0: Setup and Imports

First, let's grab the tools we need. Run the cell below. If you get an error, you may need to install the packages via in the terminal.

`pip install numpy matplotlib plotly qutip scipy`

alternatively, in one of the cells, type:

`!pip install numpy matplotlib plotly qutip scipy`


In [ ]:
!pip install numpy matplotlib plotly qutip scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.special import eval_hermite, factorial
from scipy.linalg import expm
import qutip as qt

# Setting up a nice theme for plots
plt.style.use('seaborn-v0_8-darkgrid')

## Part 1: Numpy Essentials for Quantum Mechanics

Quantum Mechanics is effectively "Linear Algebra with fancy notation". To simulate it, we need a library that handles vectors and matrices efficiently. Enter **NumPy**.

In QM, a state $|\psi\rangle$ is a vector. Operators like $\hat{x}$ or $\hat{H}$ are matrices.

### Challenge 1.1: Vectors and Inner Products
Create two vectors, `v1` and `v2`. Calculate their inner product (dot product). In bra-ket notation, this is $\langle v1 | v2 \rangle$.

In [ ]:
# EXERCISE: Create two numpy arrays representing vectors
# v1 = np.array([ ... ])
# v2 = ...
# Calculate dot product using np.dot(v1, v2) or v1 @ v2

# Your code here:


--- 
## Part 2: The Quantum Harmonic Oscillator Refresher

Recall the Hamiltonian for a harmonic oscillator:
$$\hat{H} = \hbar\omega \left(\hat{a}^\dagger \hat{a} + \frac{1}{2}\right)$$

The operators $\hat{a}$ (annihilation) and $\hat{a}^\dagger$ (creation) act on the number states $|n\rangle$ as:
$$ \hat{a}|n\rangle = \sqrt{n} |n-1\rangle $$
$$ \hat{a}^\dagger|n\rangle = \sqrt{n+1} |n+1\rangle $$

We can't simulate an *infinite* Hilbert space on a computer. So we **truncate** it. We pick a maximum number of photons, say $N=20$. 
Our basis states are $|0\rangle, |1\rangle, \dots, |N-1\rangle$.

## Part 3: Matrix Mechanics

Let's build these operators from scratch as matrices. 

The annihilation operator $\hat{a}$ in the number basis has elements:
$$ \hat{a}_{mn} = \langle m | \hat{a} | n \rangle = \sqrt{n} \delta_{m, n-1} $$

This means the matrix has $\sqrt{1}, \sqrt{2}, \dots$ on the first upper diagonal.

In [ ]:
# EXERCISE: Construct the annihilation operator matrix
N = 20  # Dimension of our Hilbert space

def create_annihilation_op(dim):
    # Hint: np.diag can create a matrix with a given diagonal
    # The diagonal we want is k=1 (upper diagonal)
    # Values are sqrt(1), sqrt(2), ..., sqrt(dim-1)
    pass

# a_op = create_annihilation_op(N)
# print(a_op)

### Challenge 3.2: The Hamiltonian and Commutation
Using your `a_op`, create the creation operator `adag_op` (it's just the conjugate transpose!). Then construct $\hat{H}$.

Also, check the commutation relation: $[\hat{a}, \hat{a}^\dagger] = \mathbb{I}$. 
**Question:** Does it hold exactly given we truncated the space?

In [ ]:
# EXERCISE
# 1. Define adag_op using .conj().T
# 2. Calculate H = hbar_omega * (adag @ a + 0.5 * Identity)
#    Assume hbar * omega = 1 for simplicity
# 3. Calculate commutator C = a @ adag - adag @ a
# 4. Check if C is equal to Identity (np.eye(N)). Where does it fail?

# Your code here:

### Challenge 3.3: Finding Eigenenergies
Now, use `np.linalg.eigh` (Hermitian eigendecomposition) to find the eigenvalues of $\hat{H}$. 
You should expect to see $0.5, 1.5, 2.5, \dots$.

In [ ]:
# EXERCISE: Compute eigenvalues of H
# evals, evecs = np.linalg.eigh(H)
# print("First 5 energies:", evals[:5])

--- 
## Part 4: Visualizing Wavefunctions (Position Space)

Matrix mechanics is abstract. Let's see what these states look like in real space $x$.
The analytic solution is:
$$ \psi_n(x) = \frac{1}{\sqrt{2^n n! \sqrt{\pi}}} e^{-x^2/2} H_n(x) $$
where $H_n(x)$ are Hermite polynomials.

In [ ]:
# EXERCISE: Implement the analytic wavefunction
def psi_n_analytic(x, n):
    # Use eval_hermite(n, x)
    # Don't forget the normalization constant!
    pass

x_vec = np.linspace(-5, 5, 200)
# Plot the first 3 states (|0>, |1>, |2>) using matplotlib
# plt.plot(x_vec, psi_n_analytic(x_vec, 0))

### Interactive Exploration with Plotly
Static plots are okay, but interactive ones are better. Let's make a plot where you can toggle traces.

In [ ]:
# DEMO: Interactive Plotly Graph
fig = go.Figure()

for n in range(4):
    y_vals = psi_n_analytic(x_vec, n)
    # We often plot |psi|^2 for probability density, or just psi + Energy offset
    fig.add_trace(go.Scatter(x=x_vec, y=y_vals, name=f'n={n}'))

fig.update_layout(title="Harmonic Oscillator Eigenstates", hovermode="x")
fig.show()

---
## Part 5: Time Evolution

If we start in a superposition state $|\Psi(0)\rangle$, how does it evolve?
$$ |\Psi(t)\rangle = \hat{U}(t) |\Psi(0)\rangle = e^{-i\hat{H}t/\hbar} |\Psi(0)\rangle $$

Since we calculate $\hat{H}$ as a matrix, we can just exponentiate it!

In [ ]:
# EXERCISE: Evolve a superposition state
# 1. Define initial state psi0 = (|0> + |1>)/sqrt(2)
# 2. Compute U(t) for t = 2*pi (one full period)
# 3. Compute psi_final = U @ psi0

# Hint: scipy.linalg.expm computes matrix exponential

---
## Part 6: Numerical Methods for Schrödinger Equation

What if the potential wasn't $V(x) = \frac{1}{2}x^2$? The ladder operator trick might not work. 
We can solve the differential equation directly using the **Finite Difference Method**.

We discretize space into points $x_i$ and approximate the second derivative:
$$ \frac{d^2\psi}{dx^2} \approx \frac{\psi_{i+1} - 2\psi_i + \psi_{i-1}}{dx^2} $$
This turns the Kinetic Energy operator into a matrix!

In [ ]:
# EXERCISE: Finite Difference Hamiltonian
dx = x_vec[1] - x_vec[0]
N_grid = len(x_vec)

# Kinetic Energy Matrix (Tridiagonal: -2 on diagonal, 1 on off-diagonals)
# T = -0.5 * (1/dx**2) * ( ... )

# Potential Energy Matrix (Diagonal matrix with V(x) on diagonal)
# V_mat = np.diag(0.5 * x_vec**2)

# H_grid = T + V_mat
# eigenvalues_grid, _ = np.linalg.eigh(H_grid)
# print("Grid method energies:", eigenvalues_grid[:5])

---
## Part 7: The Easy Way (QuTiP)

We've done a lot of manual work. **QuTiP** is a library that does this for us.
It handles basis states, operators, and time evolution automatically.

In [ ]:
# EXERCISE: Recreate the Hamiltonian in QuTiP
# N = 20
# a = qt.destroy(N)
# H_qt = a.dag() * a + 0.5
# print(H_qt.eigenenergies()[:5])

## Conclusion
You've built a Quantum Harmonic Oscillator simulation from the ground up! You learned:
1. Representations of states and operators in Numpy.
2. How finite basis truncation affects commutation relations.
3. Analytic vs. Grid-based numerical solutions.
4. How QuTiP simplifies this workflow.

**Next Steps:** Try changing the potential in the grid method to $V(x) = x^4$ (Thinking Question: Are the levels still equally spaced?).